In [1]:
import os

import pandas as pd

from src.dataPipeline import DataPipeline

os.chdir('..')

# New Model with different data

In [2]:
from sklearn.impute import SimpleImputer, KNNImputer
import numpy as np

imputer = KNNImputer(n_neighbors=5)
#imputer = SimpleImputer(strategy='mean')

dp = DataPipeline()
df = dp.runPipeline(normalizeAndStandardize=True, imputer=imputer)
df_kaggle = dp.prepare_kaggle_dataset(filePath='./data/test_data-Kaggle-v0.11.csv', normalizeAndStandardize=True, imputer=imputer)

C:\FHNW_Programmiersachen\5_Sem\immo_challenge\src\dataPipeline.py:44: DtypeWarning: Columns (3,4,5,6,11,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,45,46,47,49,50,107,110,114,115,116,119,120,121,124,125,126,128,131,132) have mixed types. Specify dtype option on import or set low_memory=False.
  self.data = pd.read_csv(filePath)
C:\FHNW_Programmiersachen\5_Sem\immo_challenge\src\dataPipeline.py:71: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  ]].bfill(axis=1)['Space extracted']
C:\FHNW_Programmiersachen\5_Sem\immo_challenge\src\dataPipeline.py:76: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To o

In [3]:
 from src.utils.helperFunctions import pickle_dataframe, unpickle_dataframe

pickle_dataframe(df, 'df_with_type.pkl')
pickle_dataframe(df_kaggle, 'df_kaggle_with_type.pkl')


DataFrame saved to df_with_type.pkl.
DataFrame saved to df_kaggle_with_type.pkl.


In [41]:
from src.utils.helperFunctions import pickle_dataframe, unpickle_dataframe

df = unpickle_dataframe('df_without_imputer.pkl')
df_kaggle = unpickle_dataframe('df_kaggle_without_imputer.pkl')

DataFrame loaded from df_without_imputer.pkl.
DataFrame loaded from df_kaggle_without_imputer.pkl.


In [39]:
from xgboost import XGBRegressor
from sklearn.model_selection import KFold
import numpy as np
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import train_test_split



y = df['price_cleaned']
y_log = np.log(y)

X = df.drop(columns=['price_cleaned'])


train_test_split_ratio = 0.5
X_train, X_test, y_train, y_test = train_test_split(X, y_log, test_size=train_test_split_ratio)

model = XGBRegressor(objective='reg:squarederror', n_estimators=20, learning_rate=0.1, max_depth=5, max_leaves=20, reg_lambda=0.9, reg_alpha=0.9, grow_policy='lossguide')
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

y_test = np.exp(y_test)
y_pred = np.exp(y_pred)

mape = mean_absolute_percentage_error(y_test, y_pred) * 100

print(f'Test MAPE: {mape:.3f}%')

# Train Mape
y_pred = model.predict(X_train)
y_train = np.exp(y_train)
y_pred = np.exp(y_pred)

mape = mean_absolute_percentage_error(y_train, y_pred) * 100

print(f'Train MAPE: {mape:.3f}%')

Test MAPE: 35.477%
Train MAPE: 25.173%


In [41]:
from src.utils.helperFunctions import create_kaggle_results

results = model.predict(df_kaggle)
results = np.exp(results)
create_kaggle_results(results, csv_name='xgBoost_without_imputer_kaggle_results')

File xgBoost_without_imputer_kaggle_results_2024-12-20_10-43-57.csv created successfully.


C:\FHNW_Programmiersachen\5_Sem\immo_challenge\src\utils\helperFunctions.py:11: DtypeWarning: Columns (3,4,5,6,11,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,45,46,48,49,104,111,112,115,116,117,120,121,122,124,127,128,130,131) have mixed types. Specify dtype option on import or set low_memory=False.
  indexes = pd.read_csv(path_to_kaggledata)['Unnamed: 0']


In [40]:
# make list with feature names

feature_importances = pd.DataFrame()
feature_importances['feature'] = model.feature_names_in_
feature_importances['importance'] = model.feature_importances_
feature_importances = feature_importances.sort_values(by='importance', ascending=False)
feature_importances

,feature,importance
4,Space extracted,0.466767
0,detail_responsive#surface_usable,0.376785
5,Plot_area_unified,0.069855
3,gde_tax,0.044537
2,gde_population,0.023407
6,No. of rooms:,0.018648
1,ForestDensityS,0.000000
